### IMPORTACIONES

In [3]:
import pandas as pd
import json
import ast
import warnings
import pyarrow as pa
import pyarrow.parquet as pq
warnings.filterwarnings('ignore')

#### ETL SOBRE USER_REVIEWS

In [4]:
user_reviews = 'datos_json/australian_user_reviews.json'

listado = []

with open(user_reviews, 'r', encoding= 'utf-8') as archivo:
    for linea in archivo.readlines():
        listado.append(ast.literal_eval(linea))

dfUserReviews = pd.DataFrame(listado)

#Guardo en formato csv para mejorar tiempos de carga y flujo de trabajo.
dfUserReviews.to_csv('json_a_csv/australian_user_reviews.csv', index=False)

#Cargo el csv previamente creado.
userReviews = pd.read_csv("json_a_csv/australian_user_reviews.csv")

In [5]:
#Cargo el csv previamente creado.
userReviews = pd.read_csv("json_a_csv/australian_user_reviews.csv")

In [6]:
userReviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [7]:
#Reviso que no haya nulos
userReviews.isnull().any()

user_id     False
user_url    False
reviews     False
dtype: bool

In [8]:
from ast import literal_eval
#EXPLICAR QUE HICE EN CADA PASO
userReviews['reviews'] = userReviews['reviews'].apply(literal_eval)

filasReviews = userReviews.explode('reviews')

desgloseColumnas = pd.json_normalize(filasReviews['reviews'])

filasReviews = filasReviews.reset_index(drop=True)

userReviews = filasReviews.join(desgloseColumnas)

userReviews = userReviews.reset_index(drop=True)

userReviewsFinal = userReviews.drop(['user_url','reviews'], axis=1)

In [9]:
#Reviso si existen duplicados
userReviewsFinal[userReviewsFinal.duplicated()]

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
1114,bokkkbokkk,,"Posted September 24, 2015.",,346110,1 of 1 people (100%) found this review helpful,True,yep
2894,ImSeriouss,,"Posted January 10, 2014.",,218620,1 of 3 people (33%) found this review helpful,True,"Good graphics, fun heists! A bit laggy"
2895,ImSeriouss,,"Posted January 10, 2014.",,105600,0 of 2 people (0%) found this review helpful,True,So fun! DEFINITELY NOT RIP OFF OF MINECRAFT! e...
2896,ImSeriouss,,"Posted December 17, 2014.",,570,No ratings yet,True,bobo pinoy
2897,ImSeriouss,,"Posted January 13, 2014.",,211820,No ratings yet,True,If you want to play this game.. expect glithes...
...,...,...,...,...,...,...,...,...
44456,76561198092022514,,Posted July 3.,,422400,No ratings yet,True,Muy entretenido y una coleccion de armas prome...
44457,76561198092022514,,Posted June 1.,,218620,No ratings yet,True,"Tiene una jugabilidad y tematica muy buena :D,..."
44458,76561198092022514,,"Posted August 17, 2014.",,261820,No ratings yet,True,"Buen juego, no importa el desarrrollo que tien..."
44459,76561198092022514,,"Posted February 17, 2014.",,224260,No ratings yet,True,exelente aporte :D¡¡¡ es una buen mod basado e...


In [10]:
#verifico que la información disponible esté correcta para interpretar los duplicados utilizando el user_id
userReviewsFinal[userReviewsFinal['user_id']=='bokkkbokkk']

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
1113,bokkkbokkk,,"Posted September 24, 2015.",,346110,1 of 1 people (100%) found this review helpful,True,yep
1114,bokkkbokkk,,"Posted September 24, 2015.",,346110,1 of 1 people (100%) found this review helpful,True,yep


In [11]:
userReviewsFinal= userReviewsFinal.drop_duplicates()

In [12]:
userReviewsFinal[userReviewsFinal['user_id']=='bokkkbokkk']

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
1113,bokkkbokkk,,"Posted September 24, 2015.",,346110,1 of 1 people (100%) found this review helpful,True,yep


In [13]:
userReviewsFinal

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...
59328,76561198312638244,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,76561198312638244,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D


In [14]:
#se aplica la transformación del tipo en la columna item_id
userReviewsFinal['item_id'] = userReviewsFinal['item_id'].astype('Int64')

#si hay problemas eliminar los nulos y pasarlo a int

In [15]:
userReviewsFinal.isnull().any()

user_id        False
funny           True
posted          True
last_edited     True
item_id         True
helpful         True
recommend       True
review          True
dtype: bool

In [16]:
#almaceno los indices de los nulos en la columna id que no me permiten realizar la transformacion del tipo de dato
nulos = userReviewsFinal[userReviewsFinal['recommend'].isna()].index

#utilizo la variable nulos donde almacene el indice de nulos para ubicar las filas correspondientes
userReviewsFinal.iloc[nulos]

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
137,gdxsd,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
177,76561198094224872,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
2560,76561198074079694,,"Posted October 17, 2015.",,208090,No ratings yet,True,Best game with gore. =P
10205,PintoProAlto,1 person found this review funny,"Posted April 21, 2014.",,570,3 of 3 people (100%) found this review helpful,True,belezura d jogo
14008,76561198064975700,,"Posted June 26, 2012.",,440,No ratings yet,True,'tis good
15777,mtmoss,,"Posted January 9, 2014.",,252490,1 of 1 people (100%) found this review helpful,True,"Rust is stil in uts early stages. For now, i h..."
19609,zrustz16,,"Posted July 7, 2014.",,304930,4 of 6 people (67%) found this review helpful,True,"Jogo divertido, graficos quadrados s2, tem um ..."
20664,WardPearce,,Posted March 18.,,373360,3 of 5 people (60%) found this review helpful,True,"Very, Very Fun. One of the best games I have e..."
25593,BM5K,,"Posted July 7, 2014.",,242920,No ratings yet,True,If you're into civilization-building (note the...
26815,matrix12384,,"Posted May 24, 2013.","Last edited January 1, 2014.",570,No ratings yet,True,"Dota 2, hours of fun spent playing this game w..."


In [17]:
#luego de analizar el dataset obtenido a partir de los nulos se procede a eliminar las filas 137 y 177 ya que las componen un 90% de datos nulos.
userReviewsFinal.drop(index=137,inplace=True)
userReviewsFinal.drop(index=177,inplace=True)

In [18]:
#se procede a eliminar las columnas funny, Last_edited y helpful ya que no serán usadas por la API ni por el Sentiment Analysis
userReviewsFinal = userReviewsFinal.drop(['funny','last_edited','helpful'], axis= 1)

In [19]:
userReviewsFinal

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,70,True,a must have classic from steam definitely wort...
59329,76561198312638244,Posted July 8.,362890,True,this game is a perfect remake of the original ...
59330,LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,Posted July 20.,730,True,:D


In [20]:
#Extraigo el año de la columna posted a fin de utilizarlo
#userReviewsFinal['review_year'] = userReviewsFinal['posted'].str.extract(r'(\b\d{4}\b)')

#userReviewsFinal['review_year'] = userReviewsFinal['review_year'].astype('Int64')

userReviewsFinal["posted"] = pd.to_datetime(userReviewsFinal["posted"].astype(str).str.replace(r'Posted |,|\.', '', regex=True), errors='coerce')

In [21]:
userReviewsFinal['posted'] = userReviewsFinal['posted'].interpolate(method='linear', limit_area='inside')

In [22]:
userReviewsFinal.dtypes

user_id              object
posted       datetime64[ns]
item_id               Int64
recommend            object
review               object
dtype: object

In [23]:
userReviewsFinal['review_year'] = userReviewsFinal['posted'].dt.year

userReviewsFinal

,user_id,posted,item_id,recommend,review,review_year
0,76561197970982479,2011-11-05,1250,True,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,2011-07-15,22200,True,It's unique and worth a playthrough.,2011
2,76561197970982479,2011-04-21,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011
3,js41637,2014-06-24,251610,True,I know what you think when you see this title ...,2014
4,js41637,2013-09-08,227300,True,For a simple (it's actually not all that simpl...,2013
...,...,...,...,...,...,...
59328,76561198312638244,2015-12-29,70,True,a must have classic from steam definitely wort...,2015
59329,76561198312638244,2015-12-29,362890,True,this game is a perfect remake of the original ...,2015
59330,LydiaMorley,2015-12-29,273110,True,had so much fun plaing this and collecting res...,2015
59331,LydiaMorley,2015-12-29,730,True,:D,2015


In [24]:
userReviewsFinal.drop('posted', axis=1, inplace=True)

### Sentiment Analyzer

Al dataframe de User Reviews se le va a realizar un analisis de sentimiento para poder determinar en base a un modelo estadistico 

##### Importaciones

In [25]:
import nltk
nltk.download('vader_lexicon')

from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to C:\Users\Nano
[nltk_data]     L3AN\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [26]:
#se procede a normalizar el texto a utilizar en el analisis de sentimiento.

userReviewsFinal['review'] = userReviewsFinal['review'].str.lower()

In [27]:
#obtengo los indices de los nulos y los traigo con iloc para realizar una lectura y analisis correspondientes
reviewsNulos = userReviewsFinal[userReviewsFinal['review'].isnull()].index

userReviewsFinal.iloc[reviewsNulos]

,user_id,item_id,recommend,review,review_year
2562,Halio99,730,True,you either die a noob; repetitively. or live l...,2015
10207,PintoProAlto,4000,True,melhor que pinto no lixo,2014
14011,H4unted_P3n,440,True,team fortress 2 is a excellent fps game that i...,2014
15779,mtmoss,286100,True,quite a good fun little game for a free to pla...,2014
19611,zrustz16,265630,True,muito bom !,2014
20666,WardPearce,319630,True,this is one the best game i have played.the be...,2014
25595,BZ4LIFE,407250,False,aweful gameplay; players take 2 seconds to res...,2014
26817,76561198052485823,99900,True,that's right free to play!and have funny to yo...,2013
27254,roitethere,224260,True,"what more could you ask for a zombie game, it ...",2015
29046,SkateboardSquats,730,True,step 1: play gamestep 2: get skinsstep 3: ????...,2014


In [28]:
userReviewsFinal

,user_id,item_id,recommend,review,review_year
0,76561197970982479,1250,True,simple yet with great replayability. in my opi...,2011
1,76561197970982479,22200,True,it's unique and worth a playthrough.,2011
2,76561197970982479,43110,True,great atmosphere. the gunplay can be a bit chu...,2011
3,js41637,251610,True,i know what you think when you see this title ...,2014
4,js41637,227300,True,for a simple (it's actually not all that simpl...,2013
...,...,...,...,...,...
59328,76561198312638244,70,True,a must have classic from steam definitely wort...,2015
59329,76561198312638244,362890,True,this game is a perfect remake of the original ...,2015
59330,LydiaMorley,273110,True,had so much fun plaing this and collecting res...,2015
59331,LydiaMorley,730,True,:d,2015


In [29]:
userReviewsFinal['review'] = userReviewsFinal['review'].astype(str)

In [30]:
def puntaje(texto):
    if texto:
        compound_score = sia.polarity_scores(texto)['compound']
        if compound_score >= 0.05:
            return 2  # positivo
        elif compound_score <= -0.05:
            return 0  # negativo
        else:
            return 1  # neutral
    else:
        return 1  # valor por defecto si no hay reseña

userReviewsFinal['sentiment_analysis'] = userReviewsFinal['review'].apply(puntaje)
userReviewsFinal.drop('review', axis=1, inplace=True)  # Eliminar la columna original 'review'

In [31]:
userReviewsFinal[userReviewsFinal['sentiment_analysis'] == 2]

,user_id,item_id,recommend,review_year,sentiment_analysis
0,76561197970982479,1250,True,2011,2
1,76561197970982479,22200,True,2011,2
2,76561197970982479,43110,True,2011,2
3,js41637,251610,True,2014,2
4,js41637,227300,True,2013,2
...,...,...,...,...,...
59328,76561198312638244,70,True,2015,2
59329,76561198312638244,362890,True,2015,2
59330,LydiaMorley,273110,True,2015,2
59331,LydiaMorley,730,True,2015,2


In [32]:
#realizo una comprobación con los respectivos puntajes para ver si devuelve valores correctos

#userReviewsFinal.loc[userReviewsFinal['sentiment_analysis'] == 2, 'item_id']


In [33]:
# Para evitar problemas futuros en la interpretación de datos al cargar el archivo csv se procede a eliminar los nulos en la columna item_id
userReviewsFinal[userReviewsFinal['item_id'].isna() == True].count()

user_id               26
item_id                0
recommend              0
review_year           26
sentiment_analysis    26
dtype: int64

In [34]:
userReviewsFinal["item_id"].fillna(0, inplace=True)

In [35]:
userReviewsFinal[userReviewsFinal['item_id'] == 0]

,user_id,item_id,recommend,review_year,sentiment_analysis
2559,76561198021575394,0,NaN,2015,1
10080,cmuir37,0,NaN,2015,1
13767,Jaysteeny,0,NaN,2014,1
15493,ML8989,0,NaN,2015,1
19184,76561198079215291,0,NaN,2014,1
20223,76561198079342142,0,NaN,2014,1
25056,76561198061996985,0,NaN,2015,1
26257,76561198108286351,0,NaN,2014,1
26674,CallTripleZero,0,NaN,2014,1
28444,Priceless612,0,NaN,2015,1


In [36]:
userReviewsFinal.count()


user_id               58457
item_id               58457
recommend             58431
review_year           58457
sentiment_analysis    58457
dtype: int64

In [37]:
userReviewsFinal = userReviewsFinal[userReviewsFinal['item_id'] != 0]

In [38]:
userReviewsFinal.rename(columns={'item_id': 'id'}, inplace=True)

In [39]:
userReviewsFinal.isna().mean()

user_id               0.0
id                    0.0
recommend             0.0
review_year           0.0
sentiment_analysis    0.0
dtype: float64

In [40]:
userReviewsFinal.to_csv('csv_limpios/user_reviews.csv', index= False)

In [41]:
# Se convierte el DataFrame a una tabla de Arrow para posteriormente comprimirlo en parquet
table = pa.Table.from_pandas(userReviewsFinal)

# Especifica el nombre del archivo Parquet y comprime con snappy (puedes elegir otros algoritmos)
parquet_file = 'datos_parquet/user_reviews.parquet'
pq.write_table(table, parquet_file, compression='snappy')